In [13]:
import numpy as np
import pandas as pd

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
path = '/content/drive/MyDrive/tamtam/제주도음식점리뷰포함(번역).csv'
restaurants = pd.read_csv(path, encoding='utf-8')

In [17]:
restaurants['리뷰'] = restaurants['번역된 리뷰']

In [18]:
restaurants = restaurants.drop(['번역된 리뷰'], axis=1)

In [19]:
data = restaurants.copy()

In [21]:
data['리뷰'] = data['리뷰'].str.replace('\n', '', regex=False)

In [22]:
data

,이름,주소,평점,위도,경도,카테고리,리뷰
0,양씨네 제주전복칼국수 본점,대한민국 제주특별자치도 제주시 관덕로13길 10-1 1층,4.8,33.513904,126.526098,"식당, 명소, 음식,",우리는 Google지도 에서이 식당을 우연히 발견했고 즐겁게 놀랐습니다!맛있는 타르...
1,제주흑우레스토랑,대한민국 제주특별자치도 제주시 아란서길 110,5.0,33.468196,126.539946,"식당, 명소, 음식,",매우 즐거운 저녁 식사였습니다.Jeju Black Cow Beef는 정말 맛있습니다...
2,은희네해장국 본점,대한민국 제주특별자치도 제주시 일도2동 357-4,4.3,33.508523,126.540804,"식당, 명소, 음식,",NaN
3,똣똣라면 본점,"대한민국 제주특별자치도 제주시 특별자치도, 한림읍 금악로 18",4.3,33.358697,126.296666,"식당, 명소, 음식,",좋은Alley Restaurant을 라이브로 보았던 남편과 저는 Jeju Islan...
4,종가전복(Jong Ga Abalone Restaurant),"대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 해맞이해안로 1296",4.9,33.531892,126.851134,"식당, 명소, 음식,",환상적인 전복 쌀과 죽 !!너무 신선하고 맛이 좋습니다.우리는 또한 매우 맛있고 강...
...,...,...,...,...,...,...,...
5511,돈주는빠식이,대한민국 제주특별자치도 제주시 조천읍 신북로 584-6,4.7,33.539559,126.671786,"식당, 명소, 음식,",NaN
5512,플로라 레스토랑,대한민국 제주특별자치도 제주시 조천읍 신북로 502,0.0,33.542502,126.666179,"식당, 명소, 음식,",리뷰 없음
5513,정민이네 갈치 월정본점(正民之家带鱼 月汀里本店) 갈치요리 전문점 (银带鱼料理专门店)...,"대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 월정3길 53-17 2층",5.0,33.556495,126.794382,"식당, 음식, 명소,",직원들은 매우 친절합니다바다의 열린 전망으로 식사를 할 수있어서 반갑습니다.나는 모...
5514,해신짬뽕,"대한민국 제주특별자치도 제주시 특별자치도, 우도면 우도해안길 814",4.4,33.513337,126.957563,"식당, 음식, 명소,","나는 해산물 국수 수프, 검은 콩 국수 및 튀긴 만두를 먹었다.모두가 맛있고 바다 ..."


In [23]:
# NaN과 '리뷰 없음'을 '리뷰가 없습니다.'로 통합
data["리뷰"] = data["리뷰"].replace("리뷰 없음", None)  # '리뷰 없음'을 NaN으로 변환
data["리뷰"] = data["리뷰"].fillna("리뷰가 없습니다.")   # NaN을 기본값으로 대체


In [24]:
def refine_category_with_more_keywords(row):
    """
    음식점 이름과 리뷰를 기반으로 다양한 경우를 고려한 카테고리 생성
    :param row: 데이터프레임의 한 행
    :return: 생성된 카테고리
    """
    # 안전하게 문자열로 변환
    name = str(row['이름']) if isinstance(row['이름'], str) else ""
    review = str(row['리뷰']) if isinstance(row['리뷰'], str) else ""

    # 카테고리 분류
    if any(keyword in name or keyword in review for keyword in ["전복", "해녀", "조개"]):
        return "해산물/전복"
    elif any(keyword in name or keyword in review for keyword in ["갈치", "고등어", "생선구이", "바다", "해물", "횟집", "회"]):
        return "해산물/생선구이"
    elif any(keyword in name or keyword in review for keyword in ["고기", "갈비", "육류", "숯불"]):
        return "고기 전문점"
    elif any(keyword in name or keyword in review for keyword in ["국수", "칼국수", "잔치국수"]):
        return "국수 전문점"
    elif any(keyword in name or keyword in review for keyword in ["국밥", "선지", "순대","해장국"]):
        return "국밥 전문점"
    elif any(keyword in name or keyword in review for keyword in ["짜장", "짬뽕", "탕수육", "중식", "중국음식"]):
        return "중식 전문점"
    elif any(keyword in name or keyword in review for keyword in ["레스토랑", "스테이크", "파스타", "양식"]):
        return "양식/이탈리안 레스토랑"
    elif any(keyword in name or keyword in review for keyword in ["피자", "마르게리타", "치즈"]):
        return "피자 전문점"
    elif any(keyword in name or keyword in review for keyword in ["치킨", "후라이드", "양념치킨", "튀김"]):
        return "치킨 전문점"
    elif any(keyword in name or keyword in review for keyword in ["커피", "카페", "디저트", "라떼", "베이커리", "빵"]):
        return "카페/디저트"
    elif any(keyword in name or keyword in review for keyword in ["샤브샤브", "탕", "찜", "국물"]):
        return "한식/국물 요리"
    elif any(keyword in name or keyword in review for keyword in ["뷔페", "무한리필"]):
        return "뷔페/무한리필"
    elif any(keyword in name or keyword in review for keyword in ["분식", "떡볶이", "김밥", "어묵"]):
        return "분식 전문점"
    elif any(keyword in name or keyword in review for keyword in ["한식", "비빔밥", "된장찌개", "불고기"]):
        return "한식 전문점"
    elif any(keyword in name or keyword in review for keyword in ["패스트푸드", "버거", "햄버거", "프렌치프라이"]):
        return "패스트푸드"
    elif any(keyword in name or keyword in review for keyword in ["채식", "비건", "샐러드"]):
        return "채식 전문점"
    else:
        return "기타"


In [27]:
data['카테고리'] = "restaurants"

In [28]:
# 개선된 카테고리 생성 함수 적용
data['세분화된 카테고리'] = data.apply(refine_category_with_more_keywords, axis=1)
data

,이름,주소,평점,위도,경도,카테고리,리뷰,세분화된 카테고리
0,양씨네 제주전복칼국수 본점,대한민국 제주특별자치도 제주시 관덕로13길 10-1 1층,4.8,33.513904,126.526098,restaurants,우리는 Google지도 에서이 식당을 우연히 발견했고 즐겁게 놀랐습니다!맛있는 타르...,해산물/전복
1,제주흑우레스토랑,대한민국 제주특별자치도 제주시 아란서길 110,5.0,33.468196,126.539946,restaurants,매우 즐거운 저녁 식사였습니다.Jeju Black Cow Beef는 정말 맛있습니다...,고기 전문점
2,은희네해장국 본점,대한민국 제주특별자치도 제주시 일도2동 357-4,4.3,33.508523,126.540804,restaurants,리뷰가 없습니다.,국밥 전문점
3,똣똣라면 본점,"대한민국 제주특별자치도 제주시 특별자치도, 한림읍 금악로 18",4.3,33.358697,126.296666,restaurants,좋은Alley Restaurant을 라이브로 보았던 남편과 저는 Jeju Islan...,고기 전문점
4,종가전복(Jong Ga Abalone Restaurant),"대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 해맞이해안로 1296",4.9,33.531892,126.851134,restaurants,환상적인 전복 쌀과 죽 !!너무 신선하고 맛이 좋습니다.우리는 또한 매우 맛있고 강...,해산물/전복
...,...,...,...,...,...,...,...,...
5511,돈주는빠식이,대한민국 제주특별자치도 제주시 조천읍 신북로 584-6,4.7,33.539559,126.671786,restaurants,리뷰가 없습니다.,기타
5512,플로라 레스토랑,대한민국 제주특별자치도 제주시 조천읍 신북로 502,0.0,33.542502,126.666179,restaurants,리뷰가 없습니다.,양식/이탈리안 레스토랑
5513,정민이네 갈치 월정본점(正民之家带鱼 月汀里本店) 갈치요리 전문점 (银带鱼料理专门店)...,"대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 월정3길 53-17 2층",5.0,33.556495,126.794382,restaurants,직원들은 매우 친절합니다바다의 열린 전망으로 식사를 할 수있어서 반갑습니다.나는 모...,해산물/생선구이
5514,해신짬뽕,"대한민국 제주특별자치도 제주시 특별자치도, 우도면 우도해안길 814",4.4,33.513337,126.957563,restaurants,"나는 해산물 국수 수프, 검은 콩 국수 및 튀긴 만두를 먹었다.모두가 맛있고 바다 ...",해산물/생선구이


In [29]:
path = '/content/drive/MyDrive/tamtam/제주도 음식점 크롤링.csv'
data.to_csv(path, index=False)

In [ ]:
# 결과 확인
data.head()

,이름,주소,평점,위도,경도,카테고리,리뷰,세분화된 카테고리,ID
0,양씨네 제주전복칼국수 본점,대한민국 제주특별자치도 제주시 관덕로13길 10-1 1층,4.8,33.513904,126.526098,"식당, 명소, 음식,",우리는 Google지도 에서이 식당을 우연히 발견했고 즐겁게 놀랐습니다!\n\n맛있...,해산물/전복,cf6372b0b2a8766027051e57f441d466
1,제주흑우레스토랑,대한민국 제주특별자치도 제주시 아란서길 110,5.0,33.468196,126.539946,"식당, 명소, 음식,",매우 즐거운 저녁 식사였습니다.Jeju Black Cow Beef는 정말 맛있습니다...,고기 전문점,73635851ec9e54f2086c1d86628c3d41
2,은희네해장국 본점,대한민국 제주특별자치도 제주시 일도2동 357-4,4.3,33.508523,126.540804,"식당, 명소, 음식,",리뷰가 없습니다.,국밥 전문점,5493d4e7a55f271cd7b22d247d2ee970
3,똣똣라면 본점,"대한민국 제주특별자치도 제주시 특별자치도, 한림읍 금악로 18",4.3,33.358697,126.296666,"식당, 명소, 음식,",좋은\nAlley Restaurant을 라이브로 보았던 남편과 저는 Jeju Isl...,고기 전문점,3feaf7f032448abbc01f034c307e5d1e
4,종가전복(Jong Ga Abalone Restaurant),"대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 해맞이해안로 1296",4.9,33.531892,126.851134,"식당, 명소, 음식,",환상적인 전복 쌀과 죽 !!너무 신선하고 맛이 좋습니다.우리는 또한 매우 맛있고 강...,해산물/전복,71a0dfa62211be05c864c7ddc6c1ea10


In [ ]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
pip install pinecone-client sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.0.0
    Uninstalling pinecone-plugin-inference-3.0.0:
      Successfully uninstalled pinecone-plugin-inference-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.1 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [ ]:
api_key="*****************"

In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트 초기화
pinecone = Pinecone(api_key=api_key)

# 인덱스 이름
index_name = "restaurants"

# 인덱스 생성
if index_name not in pinecone.list_indexes().names():
    pinecone.create_index(
        name=index_name,
        dimension=384,  # 임베딩 모델 차원
        metric='cosine',  # 유사도 계산 방식
        spec=ServerlessSpec(
            cloud='aws',  # 클라우드 제공자
            region='us-east-1'  # 리전 설정
        )
    )
    print(f"인덱스 '{index_name}' 생성 완료!")
else:
    print(f"인덱스 '{index_name}'가 이미 존재합니다.")


인덱스 'restaurants' 생성 완료!


In [ ]:
from sentence_transformers import SentenceTransformer

# 임베딩 모델 로드
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pip install pinecone-client


In [ ]:
# 기존 인덱스 삭제
pinecone.delete_index("restaurants")  # 기존 인덱스 이름
print("기존 인덱스 삭제 완료!")

# 인덱스 재생성
pinecone.create_index(
    name="restaurants",
    dimension=384,  # 임베딩 차원 (임베딩 모델에 따라 설정)
    metric="cosine",  # 유사도 계산 방식
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
print("새로운 인덱스 생성 완료!")


기존 인덱스 삭제 완료!
새로운 인덱스 생성 완료!


In [ ]:
from sentence_transformers import SentenceTransformer

# 임베딩 모델 로드
model = SentenceTransformer("all-MiniLM-L6-v2")  # 또는 다른 모델 사용 가능


In [ ]:
for _, row in data.iterrows():
    # 쿼리 생성 (예: 다양한 추천 문구)
    queries = [
        f"{row['이름']}은 제주도에서 인기 있는 {row['세분화된 카테고리']}입니다.",
        f"제주도에서 {row['카테고리']}를 먹고 싶다면 {row['이름']}을 추천합니다.",
        f"{row['주소']} 근처에서 가볼만한 음식점은 {row['이름']}입니다.",
        f"평점 {row['평점']}점으로 추천되는 음식점 {row['이름']}."
    ]

    # 텍스트 필드 생성 (쿼리 포함)
    text = f"{row['이름']}은 {row['세분화된 카테고리']}으로 유명합니다.\n" + "\n".join(queries)

    # 임베딩 생성
    embedding = model.encode(text).tolist()

    # 메타데이터 구성
    metadata = {
        "name": row["이름"],
        "address": row["주소"],
        "rating": row["평점"],
        "latitude": row["위도"],
        "longitude": row["경도"],
        "category": row["세분화된 카테고리"],
        "review": row["리뷰"]
    }

    # Pinecone 업로드
    index.upsert([{
        "id": row["ID"],  # 고유 ID
        "values": embedding,  # 임베딩 벡터
        "metadata": metadata  # 메타데이터
    }])


KeyboardInterrupt: 

In [ ]:
query = "3박4일 제주도 여행일정 추천해줘 자연과 함꼐하는"
query_embedding = model.encode(query).tolist()

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    filter={
        "category": {"$eq": "양식/이탈리안 레스토랑"},
        # "rating": {"$gte": 4.0}
    }
)

for match in results["matches"]:
    if match["score"] < 0.4:  # 임계값 설정
        continue
    metadata = match["metadata"]
    print(f"추천 음식점: {metadata['name']}")
    print(f"주소: {metadata['address'], metadata['latitude'], metadata['longitude']}")
    print(f"카테고리: {metadata['category']}")
    print(f"평점: {metadata['rating']}")
    print("---")


추천 음식점: 라라코스트 서귀포 광장점
주소: ('대한민국 제주특별자치도 서귀포시 동홍동 일주동로 8670', 33.2537883, 126.5618273)
카테고리: 양식/이탈리안 레스토랑
평점: 4.0
---
추천 음식점: 봄소풍김밥
주소: ('대한민국 제주특별자치도 제주시 특별자치도, 우정로4길 4 KR 1층', 33.491449, 126.4304499)
카테고리: 양식/이탈리안 레스토랑
평점: 4.2
---
추천 음식점: 새미언덕
주소: ('대한민국 제주특별자치도 제주시 회천동 중산간동로 188-8', 33.5015725, 126.6108367)
카테고리: 양식/이탈리안 레스토랑
평점: 4.3
---
추천 음식점: 한라초밥
주소: ('대한민국 제주특별자치도 제주시 한림읍 동명리 1595', 33.4102019, 126.2652626)
카테고리: 양식/이탈리안 레스토랑
평점: 4.3
---
추천 음식점: 오니기리와이규동 제주공항점
주소: ('대한민국 제주특별자치도 제주시 공항로 2', 33.5064764, 126.4925181)
카테고리: 양식/이탈리안 레스토랑
평점: 3.0
---


In [ ]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

# Pinecone 초기화
pinecone = Pinecone(api_key=api_key)

# 인덱스 이름
index_name = "restaurants"

# 기존 인덱스에 연결
index = pinecone.Index(index_name)

# 임베딩 모델 로드
model = SentenceTransformer("all-MiniLM-L6-v2")

# 검색할 쿼리
query = "제주도 한식 맛집"

# 쿼리 임베딩 생성
query_embedding = model.encode(query).tolist()

# Pinecone 검색
results = index.query(
    vector=query_embedding,  # 쿼리 벡터
    top_k=10,  # 상위 5개 결과 반환
    include_metadata=True  # 메타데이터 포함
)

# 검색 결과 출력
for match in results["matches"]:
    metadata = match["metadata"]
    print(f"추천 음식점: {metadata['name']}")
    print(f"주소: {metadata['address']}")
    print(f"카테고리: {metadata['category']}")
    print(f"평점: {metadata['rating']}")
    print(f"리뷰: {metadata['review'][:100]}...")
    print("---")


추천 음식점: 각 is in 별
주소: 대한민국 제주특별자치도 제주시 한경면 저지리 2969-17
카테고리: 양식/이탈리안 레스토랑
평점: 5.0
리뷰: 나는 지나가고 배가 고프다 고 느끼고 있었기 때문에 Google지도에 대한 좋은 리뷰 때문에 들어갔다.
나는 디아 볼라 피자를 먹었다.
내가 돌아갈 때 자주 생각할 것 같아요.
나...
---
추천 음식점: 중문뽁순이네
주소: 대한민국 서귀포시
카테고리: 기타
평점: 5.0
리뷰: Cheonggukjang은 너무 맛있고 세트 식사는 너무 저렴하고 맛있습니다.
나는 그것을 내 자신에게 유지하고 싶지만 ... 공유하십시오....
---
추천 음식점: 못난이식당
주소: 대한민국 서귀포시
카테고리: 기타
평점: 4.3
리뷰: 리뷰가 없습니다....
---
추천 음식점: 농업회사법인(주)한스
주소: 대한민국 제주특별자치도 제주시 조천읍 함대로3길 24-9
카테고리: 해산물/생선구이
평점: 3.7
리뷰: #H2255 #H2255 브런치 #H2255 브런치 카페 #HANS 농업 회사 #Deoksan Certified Real Estate Agency
예금을주세요.내 전화, 문자 및 ...
---
추천 음식점: Cafe Terrace Korean Words
주소: 대한민국 제주특별자치도 서귀포시 안덕면 창천리 905-3
카테고리: 카페/디저트
평점: 3.5
리뷰: 좋아, 좋아, 좋은 모습이지만 커피는 그렇게 좋지 않습니다.
분위기와 견해는 모두 매우 좋습니다
나는 잠시 들었다...
---
추천 음식점: 한국중부발전 (주)제주발전본부 구내식당
주소: 대한민국 제주특별자치도 제주시 특별자치도, 삼양일동 900번지 1층 1호
카테고리: 기타
평점: 4.5
리뷰: 움직이는...
---
추천 음식점: 하도바당국수
주소: 대한민국 제주특별자치도 제주시 특별자치도, 구좌읍 하도리 1944-2 KR 번지 하도바당국수
카테고리: 고기 전문점
평점: 4.3
리뷰: 관광지의 가격을 고려할 때 상대적으로 저렴합니다.
식사에 좋습니다.
수프는 너

In [ ]:
query = "제주도에서 양식 맛집."
query_embedding = model.encode(query).tolist()

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

for match in results["matches"]:
    metadata = match["metadata"]
    print(f"추천 음식점: {metadata['name']}")
    print(f"주소: {metadata['address'], metadata['latitude'], metadata['longitude']}")
    print(f"카테고리: {metadata['category']}")
    print(f"평점: {metadata['rating']}")
    print(f"리뷰: {metadata['review'][:100]}...")
    print("---")


추천 음식점: 중문뽁순이네
주소: ('대한민국 서귀포시', 33.253718, 126.4318462)
카테고리: 기타
평점: 5.0
리뷰: Cheonggukjang은 너무 맛있고 세트 식사는 너무 저렴하고 맛있습니다.
나는 그것을 내 자신에게 유지하고 싶지만 ... 공유하십시오....
---
추천 음식점: 못난이식당
주소: ('대한민국 서귀포시', 33.2257195, 126.2567002)
카테고리: 기타
평점: 4.3
리뷰: 리뷰가 없습니다....
---
추천 음식점: 김녕스페샬
주소: ('대한민국 제주시', 33.5542052, 126.7467327)
카테고리: 기타
평점: 4.5
리뷰: 리뷰가 없습니다....
---
추천 음식점: 한끼맛있다제주시청점
주소: ('대한민국 제주시', 33.4984187, 126.530266)
카테고리: 기타
평점: 4.9
리뷰: 리뷰가 없습니다....
---
추천 음식점: 맛집올레
주소: ('대한민국 제주시', 33.521898, 126.5850602)
카테고리: 기타
평점: 4.4
리뷰: 리뷰가 없습니다....
---


In [ ]:
pip install openai==0.28


In [ ]:
import openai

openai.api_key = "***************"  # 실제 OpenAI API 키를 입력하세요


In [ ]:
from openai import ChatCompletion

# GPT API 설정
openai_api_key = openai.api_key

def generate_search_query(user_input):
    """
    사용자의 자연어 입력을 기반으로 Pinecone 검색 쿼리 생성
    """
    completion = ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 여행 계획을 위한 검색 필터 생성자 역할을 한다."},
            {"role": "user", "content": f"사용자가 입력한 문장: {user_input}. "
                                         "이 문장을 기반으로 음식점 카테고리와 필터링 조건을 반환해줘."}
        ]
    )
    return completion.choices[0].message.content


In [ ]:
pip install openai==0.28

In [ ]:
pip install openai==0.28

In [ ]:
import openai
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# OpenAI 및 Pinecone 설정
openai.api_key = openai.api_key
pinecone = Pinecone(api_key=api_key)
index = pinecone.Index("restaurants")

# GPT로 검색 조건 생성
def generate_search_query(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 검색 조건 생성 전문가야."},
            {"role": "user", "content": f"사용자가 입력한 요청: {user_input}. 검색 조건을 만들어줘."}
        ]
    )
    return response.choices[0].message["content"]

# 사용자 요청
user_input = "제주도에서 양식집 추천해줘"
gpt_response = generate_search_query(user_input)
print(f"GPT 응답: {gpt_response}")

# 쿼리 생성 및 Pinecone 검색
query = gpt_response.split("쿼리: ")[-1].split("필터: ")[0].strip()
filter_conditions = eval(gpt_response.split("필터: ")[-1].strip())
model = SentenceTransformer("all-MiniLM-L6-v2")
query_embedding = model.encode(query).tolist()

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    filter=filter_conditions
)

# 검색 결과 출력
for match in results["matches"]:
    metadata = match["metadata"]
    print(f"추천 음식점: {metadata['name']}")
    print(f"주소: {metadata['address']}")
    print(f"평점: {metadata['rating']}")
    print("---")


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
from langchain.prompts import PromptTemplate

plan_template = '''
# 템플릿 내용
여행날짜: {travel_date}, 여행기간: {travel_days}, 여행동반자: {travel_mate}, 여행테마: {travel_theme}
# 여행 계획 형식
날짜 : (날짜, 요일)
오전 : (여행계획)
오후 : (여행계획)
저녁 : (여행계획)
'''

plan_prompt = PromptTemplate(
    template=plan_template,
    input_variables=["travel_date", "travel_days", "travel_mate", "travel_theme", "theme_context"]
)


In [ ]:
travel_plan = plan_prompt.format(
    travel_date="2024-12-01",
    travel_days="3일",
    travel_mate="친구들",
    travel_theme="힐링과 맛집 탐방",
    theme_context="제주도에서 유명한 관광지와 음식점"
)


In [ ]:
# prompts.py
from langchain.prompts import PromptTemplate

plan_template = '''
# 템플릿 내용
'''

plan_prompt = PromptTemplate(
    template=plan_template,
    input_variables=["travel_date", "travel_days", "travel_mate", "travel_theme", "theme_context"]
)
